In [3]:
%pip install --quiet -U langchain-google-genai langchain langchain_core langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.3 MB/s eta 0:00:00


In [6]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [9]:
import time

index_name = "02-langchain-rag-project"  # change if desired
pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )

index = pc.Index(index_name)


In [27]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-004")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


In [30]:
vector = embeddings.embed_query("We are building RAG project")
vector

[0.02737690880894661,
 -0.0002607162168715149,
 -0.06015508994460106,
 -0.014921855181455612,
 -0.030552953481674194,
 0.01637161150574684,
 0.027066044509410858,
 -0.05008747801184654,
 0.007985391654074192,
 0.0678686872124672,
 0.042229942977428436,
 0.05927285552024841,
 -0.05352439358830452,
 -0.017972448840737343,
 0.034771181643009186,
 -0.021064143627882004,
 0.012235411442816257,
 -0.0055156610906124115,
 0.042120836675167084,
 -0.01970084011554718,
 -0.010201215744018555,
 0.04774026572704315,
 -0.0016952420119196177,
 -0.0202182587236166,
 0.006739966105669737,
 0.007446547970175743,
 0.027727458626031876,
 -0.05260198563337326,
 0.03899860754609108,
 0.020819950848817825,
 -0.031452130526304245,
 -0.0224908534437418,
 -0.07207106798887253,
 0.0011131383944302797,
 0.010975813493132591,
 -0.03834937512874603,
 0.02943241037428379,
 0.023852549493312836,
 0.003416940802708268,
 -0.014133308082818985,
 0.002638858277350664,
 -0.046862393617630005,
 -0.031954575330019,
 -0.0191

In [29]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index = index, embedding=embeddings)


In [31]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [32]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f0ca13d7-4302-456c-9fdd-f9ff140ecf95',
 '052a12af-6a65-415c-aad2-33a71950258c',
 '6a8a6442-a247-4807-9ee3-e8491f27e4d4',
 '1bdfb7dc-2097-474b-a9da-d9641584c042',
 '51181ddf-730f-4218-825e-9a0860b9a81a',
 '946fac16-e17f-469f-b7e7-622fa78fa2f9',
 '569411b9-6095-4f2e-ab18-b4ad05e773b4',
 '61858b74-304a-475b-85ab-9d25a0416997',
 'a2a7cdf7-67cb-416c-a9bb-9357e95cdf49',
 'f72a6cff-470f-4078-a53f-ded7a6c5ef28']

In [36]:
results = vector_store.similarity_search(
    "Will it be hot tomorrow",

)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model="gemini-1.5-flash")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5478920b-03ab-4c63-ac0c-36842b07d95a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [49]:
from langchain_core.messages.ai import AIMessage
abc = input("hi how may i help you today\n")
def answer_to_user(query:str):
  results=vector_store.similarity_search(query, k=2)
  context = "\n".join([doc.page_content for doc in results])
  prompt = f"Query: {query}\nContext: {context}"
  answer : AIMessage = llm.invoke(prompt)
  print(answer.content)

answer_to_user(abc)

hi how may i help you today
what is langgraph
Based on the provided context, LangGraph is a framework for building stateful, agentic applications.  It's presented as an alternative or complement to LangChain, suggesting it offers similar functionality but with a focus on statefulness and agency in the applications it creates.  More details about its specific features and capabilities are not available in the given text.

